In [1]:
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath(".."))
df = pd.read_csv("wilson_alerts.csv")
df

,time,name,ip,host,short,time_label,event_label
0,1643849106,Wazuh: ClamAV database update,10.182.193.181,mail,W-Sys-Cav,false_positive,-
1,1643849111,Wazuh: ClamAV database update,10.182.193.181,mail,W-Sys-Cav,false_positive,-
2,1643849116,Wazuh: ClamAV database update,10.182.193.181,mail,W-Sys-Cav,false_positive,-
3,1643849121,Wazuh: ClamAV database update,10.182.193.181,mail,W-Sys-Cav,false_positive,-
4,1643849126,Wazuh: ClamAV database update,10.182.193.181,mail,W-Sys-Cav,false_positive,-
...,...,...,...,...,...,...,...
634241,1644361970,AMiner: Unusual occurrence frequencies of DNS ...,192.168.239.254,inet-dns,A-Dns-Clc1,false_positive,-
634242,1644362732,AMiner: Unusual occurrence frequencies of DNS ...,192.168.239.254,inet-dns,A-Dns-Clc1,false_positive,-
634243,1644363098,AMiner: Unusual occurrence frequencies of DNS ...,192.168.239.254,inet-dns,A-Dns-Clc1,false_positive,-
634244,1644363883,AMiner: Unusual occurrence frequencies of DNS ...,192.168.239.254,inet-dns,A-Dns-Clc1,false_positive,-


In [2]:
df.rename(columns={
    'time': 'timestamp',
    'name': 'event',
    'host': 'machine',
}, inplace=True)

unique_events = df['event'].unique()
event_map = {evt: idx + 1 for idx, evt in enumerate(unique_events)}
df['event'] = df['event'].map(event_map)
unique_machines = df['machine'].unique()
machines_map = {mac: idx + 1 for idx, mac in enumerate(unique_machines)}
df['machine'] = df['machine'].map(machines_map)
# Create binary label column (0=normal, 1=anomalous)
df['label'] = df['event_label'].apply(lambda x: 0 if x == '-' else 1)

df.drop(columns=['event_label', 'time_label', 'short', 'ip'], inplace=True)

df.sort_values(by=['timestamp'], inplace=True)
min_timestamp = df['timestamp'].min()
df['timestamp'] = df['timestamp'] - min_timestamp
df.reset_index(drop=True, inplace=True)
df

,timestamp,event,machine,label
0,0,53,9,0
1,0,63,9,0
2,0,65,9,0
3,0,66,9,0
4,0,62,9,0
...,...,...,...,...
634241,517631,1,1,0
634242,517636,1,1,0
634243,517641,1,1,0
634244,517641,1,1,0


In [3]:
from tokenizer import tokenize

tokenize(df, save_path="./numeric_tokenizer")

Created 86 priority tokens





In [4]:
from train import train

_ = train(df, pre_processed=False, tokenizer_path="./numeric_tokenizer", num_queries_al=3)

Using device: cuda

Epoch 1/2


MLM Training: 100%|██████████| 2494/2494 [05:54<00:00,  7.04it/s, loss=1.1242, acc=0.7667, lr=2.50e-05]


Train Loss: 1.3446, Train Acc: 0.6466

Epoch 2/2


MLM Training: 100%|██████████| 2494/2494 [05:49<00:00,  7.13it/s, loss=1.3360, acc=0.5862, lr=0.00e+00]


Train Loss: 1.1677, Train Acc: 0.6589
Best loss: 1.1677
loaded 39 tensors
Model saved to ./pretrained_classifier


...

Iteration #0 (200 samples)

Train accuracy: 1.00000
Test accuracy: 0.99730
Test F1 score: 0.99802
Confusion Matrix (Test):
[[201717   1713]
 [     9 434793]]


100%|██████████| 638232/638232 [04:21<00:00, 2439.21it/s]


Iteration #1 (300 samples)

Train accuracy: 1.00000
Test accuracy: 0.99915
Test F1 score: 0.99938
Confusion Matrix (Test):
[[202916    514]
 [    27 434775]]


100%|██████████| 638232/638232 [02:08<00:00, 4972.43it/s]


Iteration #2 (400 samples)

Train accuracy: 1.00000
Test accuracy: 0.99904
Test F1 score: 0.99929
Confusion Matrix (Test):
[[202836    594]
 [    21 434781]]
